In [0]:
# create a sinple data frame and save it as a delta format file

df = spark.createDataFrame([(1, 21),(2, 30),(3, 49),(4, 50),(5, 99)], ['id', 'age'])
df.write.format("delta").save("/tmp/delta/people")

In [0]:
dbutils.fs.ls("/tmp/delta/people/_delta_log/")

In [0]:
# load csv file from DBFS 
df = spark.read.format("csv").option("header", "true").load("/FileStore/tables/weather.csv")

In [0]:
df.printSchema()

In [0]:
# [DELTA_INVALID_CHARACTERS_IN_COLUMN_NAMES] Found invalid character(s) among ' ,;{}()\n\t=' in the column names of your schema.
# Invalid column names: `Date.Week of`.
df.write.format("delta").save("/tmp/delta/weather2")

In [0]:
df = df.withColumnRenamed("Date.Week of", "Date_Week_of").withColumnRenamed("Data.Temperature.Avg Temp", "Data_Temperature_Avg_Temp").withColumnRenamed("Data.Temperature.Max Temp", "Data_Temperature_Max_Temp").withColumnRenamed("Data.Temperature.Min Temp", "Data_Temperature_Min_Temp")

df.write.format("delta").save("/tmp/delta/weather")

In [0]:
# read data from "/tmp/delta/people"

df = spark.read.format("delta").load("/tmp/delta/people")
df.show()

In [0]:
display(df.history())



from delta.tables import DeltaTable
 
deltaTable = DeltaTable.forPath(spark, "abfss://mycontainer@testcresoldemo.dfs.core.windows.net/md2")
# Show history
display(deltaTable.history())

 
%sql
select * from trainingcatalog.mydemoschema.deltatablefromcsv1 TIMESTAMP AS OF '2025-07-11T05:13:06.000+00:00';

df_v1 = spark.read.format("delta") \
  .option("versionAsOf", 0) \
  .load("abfss://mycontainer@testcresoldemo.dfs.core.windows.net/md2")
 
display(df_v1)
 
df_v1 = spark.read.format("delta") \
  .option("timestampAsOf", "2025-07-11T05:25:16.000+00:00") \
  .load("abfss://mycontainer@testcresoldemo.dfs.core.windows.net/md2")
 
display(df_v1)

%sql
-- Step 1: Create temp view from old version
CREATE OR REPLACE TEMP VIEW rollback_view AS
SELECT * FROM trainingcatalog.mydemoschema.deltatablefromcsv1 VERSION AS OF 0;
 
-- Step 2: Overwrite current table
INSERT OVERWRITE TABLE trainingcatalog.mydemoschema.deltatablefromcsv1
SELECT * FROM rollback_view;
 

In [0]:
data_target = [(1, "Alice"), (2, "Bob")]
df_target = spark.createDataFrame(data_target, ["id", "name"])

data_source = [(1, "Alice", 30), (3, "Charlie", 25)]
df_source = spark.createDataFrame(data_source, ["id", "name", "age"])

df_target.display()
df_source.display()

In [0]:
# append df_source into df_target using mergeschema option
df_source.write.format("delta").mode("append").option("mergeSchema", "true").save("/tmp/delta/people04")
df = spark.read.format("delta").load("/tmp/delta/people04")
df.display()

In [0]:
df.display()

In [0]:
%sql
VACUUM '/tmp/delta/people' RETAIN 10 HOURS;

In [0]:
read_format='delta'
file_path='/databricks-datasets/learning-spark-v2/people/people-10m.delta'

In [0]:
%sql
-- read table from file_path into people01
CREATE OR REPLACE TABLE people01 AS SELECT * FROM delta.`/databricks-datasets/learning-spark-v2/people/people-10m.delta`
    
-- read table from table name

In [0]:
%sql
SELECT * FROM people01;

In [0]:
%sql
DESCRIBE people01

In [0]:
%sql
CREATE OR REPLACE TABLE people02 (
    id int,
    firstName varchar(255) NOT NULL,
    middleName varchar(255),
    lastName varchar(255),
    gender varchar(255),
    birthDate timestamp,
    ssn varchar(255),
    salary int
);

INSERT INTO people02 VALUES
(9999998, 'Billy', 'Tommy', 'Luppitt', 'M', '1992-09-17T04:00:00.000+0000', '953-38-9452', 55250),
(9999999, 'Billy1', 'Tommy1', 'Luppitt1', 'M', '1992-09-17T04:00:00.000+0000', '923-38-9452', 48520),
(10000000, 'Billy2', 'Tommy2', 'Luppitt2', 'M', '1992-09-17T04:00:00.000+0000', '953-38-9452', 9000);

In [0]:
%sql
DESCRIBE people02

In [0]:
%sql
MERGE into default.people01
using default.people02
on people01.id=people02.id
when matched then update set *
WHEN NOT MATCHED THEN INSERT *

 

In [0]:
%sql
DESCRIBE people01